In [1]:
#import neccessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

2024-04-07 08:27:38.885536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 08:27:38.885665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 08:27:39.172985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df= pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
metadata_columns= ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
target_column= 'Pawpularity'
#Move last 500 images to test set
test_df= train_df.iloc[-500:]
#Delete last 500 images from train set
train_df= train_df.iloc[:-500]

#Now use last 500 images for validation
validation_df= train_df.iloc[-500:]
#Delete last 500 images from train set
train_df= train_df.iloc[:-500]

In [3]:
len(train_df), len(validation_df), len(test_df)

(8912, 500, 500)

In [4]:
#Write custom dataloader in python to load and shuffle the images and meta data
class CustomDataLoader(tf.keras.utils.Sequence):
    def __init__(self, root_dir, df, metadata_columns, target_column, batch_size, image_size, shuffle=True):
        self.root_dir = root_dir
        self.df = df
        self.metadata_columns = metadata_columns
        self.target_column = target_column
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return len(self.df) // self.batch_size #return the number of batches
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes) #shuffle the indexes after each epoch
    
    def __data_generation(self, indexes):
        X= np.empty((self.batch_size, self.image_size[0], self.image_size[1], 3))
        X_meta= np.empty((self.batch_size, len(self.metadata_columns)))
        # X_combined= []
        y= np.empty((self.batch_size, 1))
        for i, index in enumerate(indexes):
            image_path = self.root_dir + self.df.iloc[index]['Id'] + '.jpg'
            image = tf.io.read_file(image_path)
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, self.image_size)
            image = image / 255.0 #normalize the image
            image = tf.cast(image, tf.float16)
            image = tf.image.random_flip_left_right(image) #random flip left right
            X[i]= image.numpy() #convert tensor to numpy array
            X_meta[i]= self.df.iloc[index][self.metadata_columns].values
            y[i]= self.df.iloc[index][self.target_column]/100
        return X, X_meta, y
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        X, X_meta, y = self.__data_generation(indexes)
        y= tf.convert_to_tensor(y, dtype=tf.float32)
        return [X,X_meta], y

In [5]:
#Define the model with resnet50 to extract features from images
input_image = tf.keras.layers.Input(shape=(256, 256, 3))
#convert image to greyscale
# input_image = tf.keras.layers.Lambda(lambda x: tf.image.rgb_to_grayscale(x))(input_image)
input_metadata = tf.keras.layers.Input(shape=(12,))
input_image = tf.keras.layers.RandomRotation(0.15)(input_image)
# base_model = tf.keras.applications.ResNet50(include_top=False, input_tensor=input_image,
#                                             weights='imagenet')
base_model= tf.keras.applications.ResNet50(include_top=False, input_tensor=input_image,
                                            weights='imagenet')
for layer in base_model.layers:
    layer.trainable= True
x = base_model.output
x1= tf.keras.layers.GlobalAveragePooling2D()(x)
x2= tf.keras.layers.Flatten()(x1)
l1= tf.keras.layers.Dense(2048, activation='relu')(x2)
concat_layers = tf.keras.layers.Concatenate()([l1, input_metadata])
l2= tf.keras.layers.Dense(512, activation='relu')(concat_layers)
b1= tf.keras.layers.BatchNormalization()(l2)
l3= tf.keras.layers.Dense(256, activation='relu')(b1)
b2= tf.keras.layers.BatchNormalization()(l3)
# l4= tf.keras.layers.Dense(128, activation='relu')(b2)
# b3= tf.keras.layers.BatchNormalization()(l4)
l5= tf.keras.layers.Dense(64, activation='relu')(b2)
b4= tf.keras.layers.BatchNormalization()(l5)
l6= tf.keras.layers.Dense(32, activation='relu')(b4)
b5= tf.keras.layers.BatchNormalization()(l6)
l7= tf.keras.layers.Dense(16, activation='relu')(b5)
b6= tf.keras.layers.BatchNormalization()(l7)
output_1 = tf.keras.layers.Dense(1, activation='sigmoid')(b6)
#Output values between 0-100
# output= output_1 * tf.constant([100.0], dtype=tf.float64)

model = tf.keras.models.Model(inputs=[input_image,input_metadata], outputs=output_1)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# model.summary()

In [7]:
#Set dynamic learning rate to prevent overfitting
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
              loss= tf.keras.losses.MeanSquaredError(), 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [9]:
print(tf.config.list_logical_devices('GPU'))

[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')]


In [10]:
#Train the model
train_loader = CustomDataLoader('/kaggle/input/petfinder-pawpularity-score/train/', train_df, metadata_columns, target_column, 32, (256, 256))
validation_loader = CustomDataLoader('/kaggle/input/petfinder-pawpularity-score/train/', test_df, metadata_columns, target_column, 32, (256, 256))

#Custom training loop for the model
epochs= 15
train_acc_values= []
val_acc_values= []
file= open('training_log.txt', 'w')
# Define the loss function, optimizer, and metrics
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule)
train_metric = tf.keras.metrics.RootMeanSquaredError()
val_metric = tf.keras.metrics.RootMeanSquaredError()

# Loop over the epochs
for epoch in range(epochs):
    print(f"Start of epoch {epoch}")
    file.write(f"Start of epoch {epoch}\n")
    #Iterate over the batches of dataset
    for i, X in enumerate(train_loader):
        X_images= X[0][0]
        X_meta= X[0][1]
        y= X[1]
        with tf.GradientTape() as tape:
            # Forward pass
            logits = model([X_images, X_meta], training=True)
            # Compute the loss value
            loss_value = loss_fn(y, logits)
        # Backward pass
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Update the weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        # Update training metric
        train_metric.update_state(y, logits)
    # Display metrics at the end of each epoch.
    train_acc = train_metric.result()
    print(f"Training acc over epoch: {train_acc}")
    file.write(f"Training acc over epoch: {train_acc}\n")
    # Reset training metrics at the end of each epoch
#     train_metric.reset_states()
    # Run a validation loop at the end of each epoch.
    for i, X in enumerate(validation_loader):
        X_images= X[0][0]
        X_meta= X[0][1]
        y= X[1]
        val_logits = model([X_images, X_meta], training=False)
        # Update val metrics
        val_metric.update_state(y, val_logits)
    val_acc = val_metric.result()
#     val_metric.reset_states()
    print(f"Validation acc: {val_acc}")
    file.write(f"Validation acc: {val_acc}\n")
    
    #Store values for plotting
    train_acc_values.append(train_acc)
    val_acc_values.append(val_acc)
file.close()
    

Start of epoch 0
Training acc over epoch: 0.2250514030456543
Validation acc: 0.20380042493343353
Start of epoch 1
Training acc over epoch: 0.21686577796936035
Validation acc: 0.20373761653900146
Start of epoch 2
Training acc over epoch: 0.21395868062973022
Validation acc: 0.20444563031196594
Start of epoch 3
Training acc over epoch: 0.21238771080970764
Validation acc: 0.20613989233970642
Start of epoch 4
Training acc over epoch: 0.21136794984340668
Validation acc: 0.2061283141374588
Start of epoch 5
Training acc over epoch: 0.2106987088918686
Validation acc: 0.21059589087963104
Start of epoch 6
Training acc over epoch: 0.2101793736219406
Validation acc: 0.2099493443965912
Start of epoch 7
Training acc over epoch: 0.20975930988788605
Validation acc: 0.2095445692539215
Start of epoch 8
Training acc over epoch: 0.20945744216442108
Validation acc: 0.21102312207221985
Start of epoch 9
Training acc over epoch: 0.2092149257659912
Validation acc: 0.21167713403701782
Start of epoch 10
Training 

In [11]:
# train_loader = CustomDataLoader('/kaggle/input/petfinder-pawpularity-score/train/', train_df, metadata_columns, target_column, 32, (100, 100))
# validation_loader = CustomDataLoader('/kaggle/input/petfinder-pawpularity-score/train/', test_df, metadata_columns, target_column, 32, (100, 100))
# history= model.fit(train_loader, validation_data= validation_loader, epochs= 2)

In [12]:
#Test the model
test_loader = CustomDataLoader('/kaggle/input/petfinder-pawpularity-score/train/', test_df, metadata_columns, target_column, 32, (256, 256))
#Custom testing loop for the model
test_acc_values= []
file= open('testing_log.txt', 'w')
# Define the loss function, optimizer, and metrics
for i, X in enumerate(test_loader):
    X_images= X[0][0]
    X_meta= X[0][1]
    y= X[1]
    logits = model([X_images, X_meta], training=False)
    #Round the values to nearest integer
#     logits= tf.math.round(logits)
    file.write(f"Predicted values: {logits}\n")
    file.write(f"Actual values: {y}\n")
    # Compute the loss value
    loss_value = loss_fn(y, logits)
    # Update training metric
    train_metric.update_state(y, logits)
# Display metrics at the end of each epoch.
test_acc = train_metric.result()
print(f"Test acc: {test_acc}")
file.write(f"Test acc: {test_acc}\n")
file.close()

Test acc: 0.20837357640266418


In [13]:
#save the model
model.save_weights('pawpularity_model.weights.h5')